In [25]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objs as go

def forecast_and_plot(ticker='AAPL', p=5, d=1, q=0, steps=10):
    # Fetch stock prices
    data = yf.download(ticker)
    prices = data['Close'].asfreq('B')  # Assuming business days. Adjust if needed.


    # Fit ARIMA model
    model = ARIMA(prices, order=(p, d, q))
    model_fit = model.fit()

    # Forecast
    forecast = model_fit.forecast(steps=steps)

    # Create a plot
    trace1 = go.Scatter(x=prices.index, y=prices, mode='lines', name='Actual Prices')
    trace2 = go.Scatter(x=forecast.index, y=forecast, mode='lines', name='Forecasted Prices', line=dict(dash='dot'))

    layout = go.Layout(title=f'{ticker} Stock Price Forecasting with ARIMA({p},{d},{q})')
    fig = go.Figure(data=[ trace1,trace2], layout=layout)

    return fig


In [26]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(id='ticker-input', value='AAPL', type='text'),
    dcc.Input(id='p-input', value='5', type='number'),
    dcc.Input(id='d-input', value='1', type='number'),
    dcc.Input(id='q-input', value='0', type='number'),
    html.Button('Submit', id='submit-button'),
    dcc.Graph(id='forecast-plot')
])

@app.callback(
    Output('forecast-plot', 'figure'),
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('ticker-input', 'value'),
     dash.dependencies.State('p-input', 'value'),
     dash.dependencies.State('d-input', 'value'),
     dash.dependencies.State('q-input', 'value')]
)
def update_graph(n_clicks, ticker, p, d, q):
    return forecast_and_plot(ticker=ticker, p=int(p), d=int(d), q=int(q))

if __name__ == '__main__':
    app.run_server(debug=True)


[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed


In [17]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA

# Download the historical stock data
def fetch_data(ticker):
    data = yf.download(ticker, start="2010-01-01", end="2023-01-01")
    return data['Close'].asfreq('B')  # Assuming business days. Adjust if needed.

# Forecast the stock prices using ARIMA
def forecast_stock(ticker):
    prices = fetch_data(ticker)
    model = ARIMA(prices, order=(5, 1, 0))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=10)
    return forecast

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Stock Price Forecasting using ARIMA"),
    dcc.Input(id='stock-ticker', type='text', placeholder='Enter Stock Ticker (e.g., AAPL)'),
    html.Button("Forecast", id='forecast-button'),
    dcc.Graph(id='forecast-graph')
])

@app.callback(
    Output('forecast-graph', 'figure'),
    [Input('forecast-button', 'n_clicks')],
    [dash.dependencies.State('stock-ticker', 'value')]
)
def update_graph(n, ticker):
    if not ticker or n is None:
        return {}
    forecast = forecast_stock(ticker)
    return {
        'data': [{'x': list(range(len(forecast))), 'y': forecast, 'type': 'line'}],
        'layout': {'title': f'Stock Price Forecast for {ticker}'}
    }

if __name__ == '__main__':
    app.run_server(debug=True)
